# Sales Chatbot Powered by Store Catalog

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import gradio as gr
import unicodedata
import re

C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configure api connections

In [2]:
load_dotenv()
openai  = os.getenv("OPENAI_API_KEY")
openai_model = "gpt-4o-mini"
openai = OpenAI()

## Product Catalog and Conversation Examples for Training

In [3]:
# Examples of successful conversations to train the model
COMPLETE_SALE = [
    {"role": "assistant", "content": "Hello, my name is Mercadito, what would you like to buy?"},
    {"role": "user", "content": "Hello Mercadito, I want some red boots"},
    {"role": "assistant", "content": "Would you prefer knee-high or ankle boots?"},
    {"role": "user", "content": "Ankle boots size 4"},
    {"role": "assistant", "content": "Great choice, the price is 350 MXN per pair"}
]

# Examples of handling unavailable products
UNAVAILABLE_SALE = [
    {"role": "assistant", "content": "Hello, my name is Mercadito, what would you like to buy?"},
    {"role": "user", "content": "Hello Mercadito, I want a denim jacket"},
    {"role": "assistant", "content": "At the moment, we only have shoes for women for all occasions in our catalog, would you like me to recommend an alternative?"},
    {"role": "user", "content": "Thanks a lot Mercadito, but I was looking for a jacket"},
    {"role": "assistant", "content": "Of course, I understand. Feel free to contact us for your next pair of boots"}
]

# Product catalog and details
PRODUCT_CONTEXT = {
    'boots': """For more context, the store sells boots:
        - Colors: red, blue, purple, yellow, black
        - Prices: 350 MXN (ankle), 500 MXN (knee-high)""",
    
    'sneakers': """For more context, the store sells sneakers:
        - 3cm heel: 200 MXN
        - 5cm heel: 270 MXN
        - 7cm heel: 500 MXN
        - comfort: 350 MXN
        - wedding: 700 MXN""",

    'sandals': """For more context, the store sells sandals:
        - platform: 450 MXN
        - low with rhinestones: 380 MXN
        - sporty: 290 MXN
        - beach: 200 MXN""",
    
    'tennis shoes': """For more context, the store sells tennis shoes:
        - running: 890 MXN
        - casual: 650 MXN
        - basketball: 950 MXN
        - training: 780 MXN"""
}


## Text Preprocessing

In [4]:
def remove_extra_spaces(text):
    """
    Replace multiple spaces with a single space.
    
    Args:
        text (str): The input string to process.
        
    Returns:
        str: The processed string with single spaces.
    """
    return re.sub(r'\s+', ' ', text).strip()

def remove_accents(text):
    """
    Remove accents and diacritical marks from a text.
    
    Args:
        text (str): The input string to process.
        
    Returns:
        str: The processed string without accents.
    """
    normalized_text = unicodedata.normalize('NFD', text)
    return ''.join(
        char for char in normalized_text if unicodedata.category(char) != 'Mn'
    )

def remove_special_characters(text):
    """
    Remove special characters, leaving only letters, numbers, and spaces.
    
    Args:
        text (str): The input string to process.
        
    Returns:
        str: The processed string with only letters, numbers, and spaces.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def lower_case(text):
    """
    Convert text to lowercase.
    
    Args:
        text (str): The input string to process.
        
    Returns:
        str: The processed string in lowercase.
    """
    return text.lower()

def preprocessing_text(text):
    """
    Preprocess text in the following order:
    1. Remove extra spaces.
    2. Remove accents.
    3. Remove special characters.
    4. Convert to lowercase.
    
    Args:
        text (str): The input string to preprocess.
        
    Returns:
        str: The fully preprocessed string.
    """
    text = remove_extra_spaces(text)
    text = remove_accents(text)
    text = remove_special_characters(text)
    text = lower_case(text)
    return text

## Chat Message Processor

In [5]:
class ChatProcessor:
    def __init__(self, system: str, examples: list, product_context: dict):
        """
        Initializes the ChatProcessor class.

        Args:
            system (str): System instructions.
            examples (list): Conversation examples.
            product_context (dict): Additional context related to products.
        """
        self.system = system
        self.examples = examples
        self.product_context = product_context

    def process_message(self, message: str, history= []):
        """
        Processes a user message and generates responses.

        Args:
            message (str): User's message.
            history (list): Conversation history.

        Yields:
            str: Real-time generated responses.
        """
        # Preprocess the message
        message = preprocessing_text(message)

        # Create initial messages
        messages = [{"role": "system", "content": self.system}] + self.examples + history + [
            {"role": "user", "content": message}
        ]

        # Add additional context if the message contains products
        for product, context in self.product_context.items():
            if product in message:
                messages.append({"role": "system", "content": context})

        return messages


## Process chat with gradio 

In [6]:
def process_chat_interface(messages, history):
    """
    Processes the conversation with Gradio.

    Args:
        messages (list): List of recent messages.
        history (list): Previous conversation history.

    Returns:
        list: Updated history with generated responses.
    """
    messages = chat_processor.process_message(messages, history)

    # Request response from the model
    stream = openai.chat.completions.create(
                model=openai_model,
                messages=messages,
                stream=True
            )

    # Process response in real-time
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


In [7]:
system_instructions = """
You are a virtual assistant for the prestigious footwear store **Mercadito Chic Shoes**, and your name is **Mercadito**.
Your goal is to help customers find the perfect pair of shoes by offering products available in our catalog: {PRODUCT_CONTEXT}.

When interacting with customers:
1. Present our options clearly and attractively, highlighting the qualities of each product.
2. At the end, mention the price of each item, emphasizing that it is **per pair of shoes**.
3. If a customer requests a product that is not in our catalog, kindly respond, expressing regret that you cannot offer that specific item. However, reassure them that you are more than willing to help them find the best alternative that fits their needs.

Remember, the customer experience should always be pleasant, trustworthy, and helpful.
"""

EXAMPLES_SALES = []
EXAMPLES_SALES.extend(COMPLETE_SALE)
EXAMPLES_SALES.extend(UNAVAILABLE_SALE)

chat_processor = ChatProcessor(
    system     = system_instructions,
    examples   = EXAMPLES_SALES,
    product_context = PRODUCT_CONTEXT
)

gr.ChatInterface(fn=process_chat_interface, type="messages").launch()


C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
